<a href="https://colab.research.google.com/github/incluirtecnologia/Marlin/blob/1.0.x/Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Resources')

Mounted at /content/drive


In [ ]:
from FaceLandmarksDataset import FaceLandmarksDataset
from CreateLandmarkDataset import CreateLandmarkDataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torchvision import datasets, models
import dlib
from PIL import Image
# Ignore warnings
import warnings
from torchvision import transforms
from skimage import io, transform

# Implementação e treinamento da rede
import torch
from torch import nn, optim

# Carregamento de Dados
from torch.utils.data import DataLoader
from torchvision import datasets


from torch import optim

# Plots e análises
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import time

#Definições


In [ ]:
# Configurando hiperparâmetros.
args = {
    'epoch_num': 150,     # Número de épocas.
    'lr': 1e-3,           # Taxa de aprendizado.
    'weight_decay': 1e-3, # Penalidade L2 (Regularização).
    'batch_size': 100,     # Tamanho do batch.
}

# Definindo dispositivo de hardware
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])
warnings.filterwarnings("ignore")



cuda


In [ ]:
def train(train_loader, net, epoch):

  # Training mode
  net.train()

  start = time.time()

  epoch_loss  = []
  pred_list, rotulo_list = [], []
  for batch in train_loader:

    dado, rotulo = batch
    rotulo = rotulo.type(torch.LongTensor)
    # # Cast do dado na GPU
    dado = dado.to(args['device'])
    rotulo = rotulo.to(args['device'])

    # Forward
    ypred = net(dado)
    loss = criterion(ypred, rotulo)
    epoch_loss.append(loss.cpu().data)

    _, pred = torch.max(ypred, axis=1)
    pred_list.append(pred.cpu().numpy())
    rotulo_list.append(rotulo.cpu().numpy())

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  epoch_loss = np.asarray(epoch_loss)
  pred_list  = np.asarray(pred_list).ravel()
  rotulo_list  = np.asarray(rotulo_list).ravel()

  acc = accuracy_score(pred_list[0], rotulo_list[0])

  end = time.time()
  print('#################### Train ####################')
  print('Epoch %d, Loss: %.4f +/- %.4f, Acc: %.2f, Time: %.2f' % (epoch, epoch_loss.mean(), epoch_loss.std(), acc*100, end-start))

  return epoch_loss.mean()

In [ ]:
def validate(test_loader, net, epoch):

  # Evaluation mode
  net.eval()

  start = time.time()

  epoch_loss  = []
  pred_list, rotulo_list = [], []
  with torch.no_grad():
    for batch in test_loader:

      dado, rotulo = batch
      rotulo = rotulo.type(torch.LongTensor)

      # Cast do dado na GPU
      dado = dado.to(args['device'])
      rotulo = rotulo.to(args['device'])

      # Forward
      ypred = net(dado)
      loss = criterion(ypred, rotulo)
      epoch_loss.append(loss.cpu().data)

      _, pred = torch.max(ypred, axis=1)
      pred_list.append(pred.cpu().numpy())
      rotulo_list.append(rotulo.cpu().numpy())

  epoch_loss = np.asarray(epoch_loss)
  pred_list  = np.asarray(pred_list).ravel()
  rotulo_list  = np.asarray(rotulo_list).ravel()

  acc = accuracy_score(pred_list[0], rotulo_list[0])

  end = time.time()
  print('********** Validate **********')
  print('Epoch %d, Loss: %.4f +/- %.4f, Acc: %.2f, Time: %.2f\n' % (epoch, epoch_loss.mean(), epoch_loss.std(), acc*100, end-start))

  return epoch_loss.mean()

#Acesso aos dados brutos

In [ ]:
EFs = [
    "afirmativa",
    # "condicional",
    # "duvida",
    # "foco",
    # "negativa",
    # "qu",
    # "relativa",
    # "s_n",
    # "topicos"
]
who = [
       'fernando',
       'felipe'
]

In [ ]:
cld = CreateLandmarkDataset()
for w in who:
  for EF in EFs:
    path = '/content/drive/MyDrive/datasets/photos/{}/{}'.format(w, EF)

    if not os.path.exists(path+'/crop'):
      if not os.path.isfile(path+'/face_landmarks.csv'):
        cld.createLandmarkDataset(root_dir=path)
      face_dataset = FaceLandmarksDataset(csv_file=path+'/face_landmarks.csv',root_dir=path) 
      cld.cropImages(root_dir=path, face_dataset=face_dataset)
      
    if not os.path.isfile(path+'/crop/face_landmarks.csv'):
      cld.createLandmarkDataset(root_dir=path+'/crop')
      print(EF, w)
    
    if os.path.isfile(path+'/crop/rotulos.txt'):
      targets = list(np.loadtxt(path+'/crop/rotulos.txt'))
    else:
      targets = list([])
    if (len(targets) < 1):
      cld.targetsCrop(root_dir=path)
      print(EF, w)

 

afirmativa fernando
afirmativa felipe


#Construíndo o input da rede

In [ ]:
datasets = []

for w in who:
  for EF in EFs:
    path = '/content/drive/MyDrive/datasets/photos/{}/{}/crop'.format(w, EF)
    transformed_dataset = FaceLandmarksDataset(csv_file=path+'/face_landmarks.csv',
                                           root_dir=path,
                                           transform=transforms.Compose([
                                               transforms.ToPILImage(),
                                               transforms.Resize(32),
                                               transforms.ToTensor()
                                           ]))
    datasets.append(transformed_dataset)
    print(w,EF,len(transformed_dataset))

auxTrain = torch.utils.data.ConcatDataset(datasets)

fernando afirmativa 1062
felipe afirmativa 847


In [ ]:
import torch.utils.data as data_utils
idx = int(len(auxTrain)*0.7)

indices_train = torch.arange(start=0, end=idx)
indices_tests = torch.arange(start= idx, end=len(auxTrain)-1)

t = data_utils.Subset(transformed_dataset, indices_train)
tt = data_utils.Subset(transformed_dataset, indices_tests)

train_loader = DataLoader(t,
                          batch_size=args['batch_size'],
                          shuffle=False,
                          drop_last=True)

test_loader = DataLoader(tt,
                          batch_size=args['batch_size'],
                          shuffle=False,
                         drop_last=True)

#Rede

In [ ]:
net = models.vgg16_bn(pretrained=True).to(args['device'])
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [ ]:
in_features = list(net.children())[-1][-1].in_features

new_classifier =list(net.classifier.children())[:-1]
new_classifier.append(nn.Linear(in_features, 2))

net.classifier = nn.Sequential(*new_classifier).to(args['device'])
print(net.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=2, bias=True)
)


In [ ]:
optimizer = optim.Adam([
            {'params': net.features.parameters(), 'lr':args['lr']*0.2, 'weight_decay': args['weight_decay']*0.2},
            {'params': net.classifier.parameters(), 'lr': args['lr'], 'weight_decay': args['weight_decay']}
        ], lr=0)
criterion = nn.CrossEntropyLoss().to(args['device'])

In [ ]:
train_losses, test_losses = [], []
for epoch in range(args['epoch_num']):
  
  # Train
  train_losses.append(train(train_loader, net, epoch))
  
  # Validate
  test_losses.append(validate(test_loader, net, epoch))

0 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
1 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
2 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
3 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
4 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
5 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
6 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
7 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
8 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
9 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
10 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
11 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
12 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
13 847 /content/drive/MyDrive/datasets/photos/felipe/afirmativa/crop
14 847 /content/drive/MyDrive/datasets/photo

IndexError: ignored

#Treinando do zero

In [ ]:
# Definindo a rede
net = nn.Sequential(
        ## ConvBlock 1
        nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),        # entrada: (b, 3, 32, 32) e saida: (b, 6, 28, 28)
        nn.BatchNorm2d(6),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=2, padding=0),           # entrada: (b, 6, 28, 28) e saida: (b, 6, 14, 14)

        ## ConvBlock 2
        nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),       # entrada: (b, 6, 14, 14) e saida: (b, 16, 10, 10)
        nn.BatchNorm2d(16),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=2, padding=0),           # entrada: (b, 16, 10, 10) e saida: (b, 16, 5, 5)

        ## ConvBlock 3
        nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0),     # entrada: (b, 16, 5, 5) e saida: (b, 120, 1, 1)
        nn.BatchNorm2d(120),
        nn.Tanh(),
        nn.Flatten(),  # lineariza formando um vetor                # entrada: (b, 120, 1, 1) e saida: (b, 120*1*1) = (b, 120)

        ## DenseBlock
        nn.Linear(360, 84),                                         # entrada: (b, 120) e saida: (b, 84)
        nn.Tanh(),
        nn.Linear(84, 2),                                          # entrada: (b, 84) e saida: (b, 10)
        )
net = net.to(args['device'])

criterion = nn.CrossEntropyLoss().to(args['device'])
optimizer = optim.Adam(net.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

In [ ]:
train_losses, test_losses = [], []
for epoch in range(args['epoch_num']):

  # Train
  train_losses.append(train(train_loader, net, epoch))

  # Validate
  test_losses.append(validate(test_loader, net, epoch))

In [ ]:
def extrai_caracteristicas(net, loader):

  # Evaluation mode
  net.eval()

  feat_list, rotulo_list = [], []
  with torch.no_grad(): 
    for k, batch in enumerate(loader):
      print('\r--{0}/{1}--'.format(k, len(loader)), end='', flush=True)

      dado, rotulo = batch

      # Cast do dado na GPU
      dado = dado.to(args['device'])
      rotulo = rotulo.to(args['device'])

      # Extração
      caracteristica = net(dado)
      feat_list.append(caracteristica.detach().cpu().numpy())
      rotulo_list.append(rotulo.detach().cpu().numpy())

  feat_list    = np.asarray(feat_list)
  feat_list    = np.reshape(feat_list, (feat_list.shape[0], -1))

  rotulo_list  = np.asarray(rotulo_list).ravel()
 
  return feat_list, rotulo_list
    

#Testando

In [ ]:
print('Load train')
train_X, train_Y = extrai_caracteristicas(net, train_loader)
print('\nLoad test')
test_X,  test_Y  = extrai_caracteristicas(net, test_loader)

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(train_X, train_Y)

pred = clf.predict(test_X)
print('{:.2f}%'.format(accuracy_score(test_Y, pred)*100))

In [ ]:
landmarks_frame = pd.read_csv(path+'/crop/face_landmarks.csv')

n = 65
img_name = landmarks_frame.iloc[n, 0]
landmarks = landmarks_frame.iloc[n, 1:]
landmarks = np.asarray(landmarks)
landmarks = landmarks.astype('float').reshape(-1, 2)

print('Image name: {}'.format(img_name))
print('Landmarks shape: {}'.format(landmarks.shape))
print('First 4 Landmarks: {}'.format(landmarks[:4]))

In [ ]:
plt.figure()
cld.show_landmarks(io.imread(img_name), landmarks)
plt.show()

In [ ]:
tl = DataLoader(auxTrain,
                          batch_size=args['batch_size'],
                          shuffle=False,
                          drop_last=True)

testl = DataLoader(auxTrain,
                          batch_size=args['batch_size'],
                          shuffle=False,
                         drop_last=True)
